In [ ]:
!pip install openai trulens-core trulens-feedback trulens-connectors-snowflake trulens-providers-cortex

In [ ]:
import pandas as pd
import numpy as np

import snowflake.core
from snowflake.snowpark import Session
from snowflake.core import Root
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session
from snowflake.cortex import complete

from typing import List
import os
import sys
import json
import time
import requests

#Set up snowflake session vars and env vars
session = get_active_session()
root = Root(session)

#Enable OpenTelemetry Tracing
os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
DB_NAME = "EMBEDDING_EVAL_DB"
SCHEMA_NAME = "DATA"
WH_NAME = "MEDIUM"

In [ ]:
oai_key = '<YOUR OAI KEY HERE>'
print('Stored API key in *oai_key* var')

In [ ]:
from openai import OpenAI
oai_client = OpenAI(api_key=oai_key)


#Access cortex search retriever built in 1st notebook
test_query = "Why might a discount code show as invalid during checkout?"


css_oai_embed = (
    root
    .databases[DB_NAME]
    .schemas[SCHEMA_NAME]
    .cortex_search_services["SUPPORT_TICKET_SEARCH_OAI_EMBED"]
)
resp = css_oai_embed.search(
    multi_index_query={
        "OAI_EMB": [
            {"vector": oai_client.embeddings.create(input=test_query,
                                                model="text-embedding-3-small").data[0].embedding}
        ],
    },    columns=["CASE_DETAILS"],
    limit=3,
)

search_results = [(row["CASE_DETAILS"]) for row in resp.results] if resp.results else []

search_results

In [ ]:
#Access cortex search retriever built in 1st notebook
test_query = "Why might a discount code show as invalid during checkout?"

css_arctic_embed = (
    root
    .databases[DB_NAME]
    .schemas[SCHEMA_NAME]
    .cortex_search_services["SUPPORT_TICKET_SEARCH_ARCTIC_EMBED"]
)
resp = css_arctic_embed.search(
    query=test_query,
    columns=["CASE_DETAILS"],
    limit=3,
)

search_results = [(row["CASE_DETAILS"]) for row in resp.results] if resp.results else []

search_results

In [ ]:
# Create the RAGWithObservability class to structure the RAG pipeline
from snowflake.cortex import complete
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes


class RAG():
    def __init__(self, llm_model, cortex_search_service_name):
        self.llm_model = llm_model
        self.cortex_search_service_name = cortex_search_service_name
        
#Here we're using the @instrument decorator to trace various stages of our RAG applicaiton

#RETRIEVEL FUNCTION

    @instrument()
    def distill_query(self, query: str):
        distilled_query = complete('llama4-maverick', f'''You are an expert query processor. Your sole function is to distill a complex 
                                user question into a minimal set of the most relevant, 
                                semantically rich keywords and key phrases suitable for a high-quality, 
                                targeted vector database search.
                                Distill the following question: {query}
                                ONLY return the key terms nothing else.
                                Make the answer extermely minimal contained 3-5 key terms and absolutely no other text''')
        return distilled_query 
        
    @instrument (
        span_type=SpanAttributes.SpanType.RETRIEVAL, 
        attributes={
            SpanAttributes.RETRIEVAL.QUERY_TEXT: "query",
            SpanAttributes.RETRIEVAL.RETRIEVED_CONTEXTS: "return",
        })  
    def retrieve_context(self, query: str):
    
        #First call cortex search service on knowledgebase!

        cortex_search_service = (
        root
        .databases[DB_NAME]
        .schemas[SCHEMA_NAME]
        .cortex_search_services[self.cortex_search_service_name])

        if self.cortex_search_service_name == "SUPPORT_TICKET_SEARCH_ARCTIC_EMBED":
            resp = cortex_search_service.search(
                query=query,
                columns=["CASE_DETAILS"],
                limit=5)
            
            search_results = [(row["CASE_DETAILS"]) for row in resp.results] if resp.results else []

        elif self.cortex_search_service_name == "SUPPORT_TICKET_SEARCH_OAI_EMBED":
            resp = cortex_search_service.search(
            multi_index_query={
                "OAI_EMB": [
                    {"vector": oai_client.embeddings.create(input=query,
                                                        model="text-embedding-3-small").data[0].embedding}
                ],
            },    
            columns=["CASE_DETAILS"],
            limit=5,
            )
            
            search_results = [(row["CASE_DETAILS"]) for row in resp.results] if resp.results else []

        else:
            print('Cant find cortex search service with that name!')
            search_results = "NONE FOUND"
            
        return search_results

#PROMPT AUGMENTATION FUNCTION

    @instrument()
    def augment_prompt(self, query: str, contexts: list) -> str:
     
        prompt = f"""
        You are an expert assistant extracting information from context provided on Customer Support Case Details.
        Answer the question based on the context. 
        Be concise and do not hallucinate.
        If you don't have the information, just say so.
        Context: {' '.join(contexts)}
        Question: {query}
        Answer:
        """
        return prompt

#COMPLETION FUNCTION

    @instrument (span_type=SpanAttributes.SpanType.GENERATION)    
    def generate_completion(self, query: str):
        
        df_response = complete(self.llm_model, query)
        return df_response

#ROOT FUNCTION
    @instrument (
        span_type=SpanAttributes.SpanType.RECORD_ROOT, 
        attributes={
            SpanAttributes.RECORD_ROOT.INPUT: "query",
            SpanAttributes.RECORD_ROOT.OUTPUT: "return",
        })
    def query_app(self, query: str) -> str:
        st.write(query)
        distilled_query = self.distill_query(query)
        contexts = self.retrieve_context(distilled_query)
        prompt = self.augment_prompt(query, contexts)
        final_response = self.generate_completion(prompt)
        st.write(final_response)
        return final_response

In [ ]:
import streamlit as st

test_query = "What are some common themes we see around billing issues"

# With web search agent disabled
rag_oai_embed = RAG(llm_model = 'claude-4-sonnet', cortex_search_service_name='SUPPORT_TICKET_SEARCH_OAI_EMBED')
rag_arctic_embed = RAG(llm_model = 'claude-4-sonnet', cortex_search_service_name='SUPPORT_TICKET_SEARCH_ARCTIC_EMBED')

#Get and print results
st.write("OAI_EMBEDDING")
response_oai_embed = rag_oai_embed.query_app(test_query)

st.write("ARCTIC_EMBEDDING")
response_arctic_embed = rag_arctic_embed.query_app(test_query)


In [ ]:
# from trulens.core import TruSession
from trulens.apps.app import TruApp
from trulens.connectors.snowflake import SnowflakeConnector

tru_snowflake_connector = SnowflakeConnector(snowpark_session=session)

app_name = "SUPPORT_TICKET_EMBEDDING_EVAL_DEMO"
version_num = 'v0'

tru_rag_oai_embed = TruApp(
    rag_oai_embed,
    app_name=app_name,
    app_version=f"OAI_EMBEDDINGS_{version_num}",
    comment = "Rag with cortex search service using OAI embeddings",
    connector=tru_snowflake_connector   
)

tru_rag_arctic_embed = TruApp(
    rag_arctic_embed,
    app_name=app_name,
    app_version=f"ARCTIC_EMBEDDINGS_{version_num}",
    comment = "Rag with cortex search service using Arctic embeddings",
    connector=tru_snowflake_connector
)

In [ ]:
import pandas as pd

prompts = [
    "What are some common themes with billing issues?",
    "What issues do we see with promo codes?",
    "What is the sentiment around issues with duplicate billing?",
    "What issues are customers seeing around shipping?",
    "How should someone resolve an issue related to payments not processing?",
    "Are we refunding customers in a timely manner?",
    "What is the top selling product we offer?",
    "What are some of the promo codes we have offered? Which ones have had issues?",
    "Who is the best player in the NBA?",
    "Are smartphones shipping on time?"
]

batch_data = pd.DataFrame({'QUERY': prompts})
batch_data

In [ ]:
from trulens.core.run import Run
from trulens.core.run import RunConfig

run_version = version_num

run_config_oai_embed = RunConfig(
    run_name=f"run_oai_embed_{run_version}",
    description="questions about snowflake AI cababilities",
    dataset_name="SNOW_RAG_DF1",
    source_type="DATAFRAME",
    label="LOCAL",
    llm_judge_name = "llama3.1-70b",
    dataset_spec={
        "RECORD_ROOT.INPUT": "QUERY",
    },
)



run_config_arctic_embed = RunConfig(
    run_name=f"run_arctic_embed_{run_version}",
    description="questions about snowflake AI cababilities",
    dataset_name="SNOW_RAG_DF1",
    source_type="DATAFRAME",
    label="LOCAL",
    dataset_spec={
        "RECORD_ROOT.INPUT": "QUERY",
    },
    
)

In [ ]:
run_oai_embed = tru_rag_oai_embed.add_run(run_config=run_config_oai_embed)
run_arctic_embed = tru_rag_arctic_embed.add_run(run_config=run_config_arctic_embed)

In [ ]:
run_oai_embed.start(input_df=batch_data)
print("Finished oai embed run")

In [ ]:
run_arctic_embed.start(input_df=batch_data)
print("Finished arctic embed run")

In [ ]:
run_list = [run_oai_embed, run_arctic_embed]

for i in run_list:
    print(f"{i.run_name} Run Status: {i.get_status()}")

In [ ]:
#The following code kicks off LLM-as-a-Judge evals for several metrics

for i in run_list:
    while i.get_status() == "INVOCATION_IN_PROGRESS":
        time.sleep(3)
    if i.get_status() == "INVOCATION_COMPLETED":
        i.compute_metrics(["coherence",
                           "answer_relevance",
                           "context_relevance",
                           "groundedness"])
        print(f"Kicked off Metrics Computation for Run {i.run_name}")
    if i.get_status() in ["FAILED", "UNKNOWN"]:
        print("Not able to compute metrics! Run status:", i.get_status())


In [ ]:
import streamlit as st

org_name = session.sql('SELECT CURRENT_ORGANIZATION_NAME()').collect()[0][0]
account_name = session.sql('SELECT CURRENT_ACCOUNT_NAME()').collect()[0][0]
db_name = session.sql('SELECT CURRENT_DATABASE()').collect()[0][0]
schema_name = session.sql('SELECT CURRENT_SCHEMA()').collect()[0][0]

st.write(f'https://app.snowflake.com/{org_name}/{account_name}/#/ai-evaluations/databases/{db_name}/schemas/{schema_name}/applications/{app_name.upper()}')

# ARCHIVE BELOW

In [ ]:
SHOW TABLES in SNOWFLAKE.LOCAL;

In [ ]:
SELECT * FROM SNOWFLAKE.LOCAL.AI_OBSERVABILITY_EVENTS;

In [ ]:
## Optional Cleanup
# for i in run_list:
#     i.delete()

In [ ]:
# query = 'what are the most common issues with billing'
# print(complete('llama4-maverick', f'''Distill the following query into its key terms: {query}.
#                                 Only return the key terms. Nothing else. 
#                                 Make the answer extermely minimal contained 3-5 key terms and absolutely no other text'''))


# print(complete('llama4-maverick', f'''You are an expert query processor. Your sole function is to distill a complex 
#                                 user question into a minimal set of the most relevant, 
#                                 semantically rich keywords and key phrases suitable for a high-quality, 
#                                 targeted vector database search.
#                                 Distill the following question: {query}
#                                 ONLY return the key terms nothing else.
#                                 Make the answer extermely minimal contained 3-5 key terms and absolutely no other text'''))